# Background Images

Melissa Graham <br>
Last verified to run Mon Feb 21, with Weekly 2021_49.

**WARNING: this Notebook is not done. The explanation for the deepCoadd background needs work.**

<br>

This is a simple tutorial on how to retrieve, display, and characterize the subtracted sky backgrounds for `calexps` and `deepCoadds`.

#### Notes

1. This tutorial assumes knowledge of afwDisplay and the Butler. Tutorials that focus on image display and the Butler can be found in this repo: https://github.com/rubin-dp0/tutorial-notebooks. (For image display, see NBs 03a, 03b, and 08a. For the Butler, see NB 04).

2. In the Butler, the `calexps` and `deepCoadds` have a `DatasetType` = `ExposureF`, and their backgrounds have `DatasetType` = `Background`. When backgrounds are retrieved, an extra step is necessary to obtain the subtracted background as an image. This will be demonstrated below.

<br>

## Set Up

#### Import packages.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter("ignore", category=UserWarning)

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib') 

#### Instantiate the Butler.

In [ ]:
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

Use the following to browse all `DatasetTypes`, included available image types and their `Backgrounds`. This is how a user would discover that `Background` is an available data type.

In [ ]:
# registry = butler.registry
# for x in sorted(registry.queryDatasetTypes()):
#     print(x)

<br>
<br>

## calexps

#### Specify visit, retrieve calexp image from the Butler.

In [ ]:
calexpId = {'visit': '703697', 'detector': 80}#, 'band': 'g'}
calexp = butler.get('calexp', dataId=calexpId)

#### Plot the calexp.

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)
plt.gca().axis('off')

#### Retrieve and plot the background that was subtracted

In [ ]:
bg_calexp = butler.get('calexpBackground', calexpId)

In [ ]:
bg_calexp_image = bg_calexp.getImage()

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.mtv(bg_calexp_image)

#### Get the array of pixel values and print statistics

Flatten the array and check for NaN values, too. Use numpy for basic array statistics.

In [ ]:
array = np.asarray( bg_calexp_image.getArray(), dtype='float' )

In [ ]:
temp = array.flatten()

In [ ]:
tx = np.where( np.isnan(temp) )[0]
print('There are ',len(tx),' NaN elements of temp.')

In [ ]:
print(len(temp))

print(np.min(temp), \
      np.max(temp), \
      np.mean(temp), \
      np.median(temp), \
      np.std(temp) )

#### Plot a histogram of the array values.

In [ ]:
plt.hist( temp, bins=20, range=(np.nanmin(temp)-0.001,np.nanmax(temp)+0.001) )
plt.axvline( np.mean(temp), color='orange' )
plt.show()

#### Clean up

In [ ]:
del temp, tx, array, bg_calexp_image, bg_calexp, calexp, calexpId

<br>
<br>

## deepCoadds

**WARNING:** The background of an image is subtracted prior to coaddition, so I'm not sure what the tiny-value uniform image that is returned by `deepCoadd_calexp_background` really is. Perhaps I'm treating it incorrectly.

#### Specify tract and patch, and retrieve deepCoadd image from the Butler.

In [ ]:
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd', coaddId)

#### Which visits went into constructing this coadd?

In [ ]:
# coaddInfo = coadd.getInfo()
# coaddVisits = coaddInfo.getCoaddInputs().visits
# coaddVisits.asAstropy()

#### Plot the deepCoadd image.

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(coadd.image)
plt.gca().axis('off')

#### Retrieve and plot the background that was subtracted

In [ ]:
bg_coadd = butler.get('deepCoadd_calexp_background', coaddId)

In [ ]:
bg_coadd_image = bg_coadd.getImage()

In [ ]:
fig = plt.figure(figsize=(10,8))
afw_display = afwDisplay.Display(1)
# afw_display.scale('linear', 'zscale')
afw_display.mtv(bg_coadd_image)

#### Get the array of pixel values and print statistics

Flatten the array and check for NaN values, too. Use numpy for basic array statistics.

In [ ]:
array = np.asarray( bg_coadd_image.getArray(), dtype='float' )

In [ ]:
temp = array.flatten()

In [ ]:
tx = np.where( np.isnan(temp) )[0]
print('There are ',len(tx),' NaN elements of temp.')

In [ ]:
print(len(temp),\
      np.min(temp), \
      np.max(temp), \
      np.mean(temp), \
      np.median(temp), \
      np.std(temp) )

#### Plot a histogram of the array values.

In [ ]:
# plt.hist( temp, bins=10, range=(np.nanmin(temp)-0.001,np.nanmax(temp)+0.001) )
# plt.show()

#### Clean up

In [ ]:
del temp, tx, array, bg_coadd_image, bg_coadd, coadd, coaddId